<a href="https://colab.research.google.com/github/armanjoy09/Image-Processing-Lab/blob/main/Experiment_No_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers

In [2]:
BATCH_SIZE = 8
IMAGE_SIZE = 128
CHANNELS = 3
EPOCHS = 10

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
"/content/drive/MyDrive/X_RAY_Dataset",
image_size= (IMAGE_SIZE, IMAGE_SIZE),
shuffle=True, seed=123, batch_size= BATCH_SIZE
)

Found 673 files belonging to 3 classes.


In [4]:
class_names=dataset.class_names
class_names

['COVID', 'NORMAL', 'PNEUMONIA']

In [5]:
len(dataset)

85

In [6]:
train_ds = dataset.take(17)
len(train_ds)

17

In [7]:
rm_ds = dataset.skip(17)
len(rm_ds)

68

In [8]:
val_ds = rm_ds.take(3)
len(val_ds)

3

In [9]:
test_ds = rm_ds.skip(3)
len(test_ds)

65

In [12]:
resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    tf.keras.layers.Rescaling(1./255),
])


In [13]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
])


In [14]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(64, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])
model.build(input_shape=input_shape)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (8, 128, 128, 3)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (8, 126, 126, 64)      │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (8, 63, 63, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (8, 61, 61, 64)        │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (8, 30, 30, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (8, 28, 28, 64)        │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (8, 14, 14, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (8, 12, 12, 64)        │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (8, 6, 6, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (8, 4, 4, 64)          │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (8, 2, 2, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (8, 256)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (8, 512)               │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (8, 3)                 │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 282,627 (1.08 MB)

 Trainable params: 282,627 (1.08 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [17]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=EPOCHS,
)

Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.2373 - loss: 1.1058 - val_accuracy: 0.2500 - val_loss: 1.1096
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 14s 820ms/step - accuracy: 0.3423 - loss: 1.0995 - val_accuracy: 0.3333 - val_loss: 1.0959
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 14s 820ms/step - accuracy: 0.3431 - loss: 1.0944 - val_accuracy: 0.3750 - val_loss: 1.0512
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 454ms/step - accuracy: 0.3612 - loss: 1.0302 - val_accuracy: 0.5000 - val_loss: 0.9460
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 11s 392ms/step - accuracy: 0.4975 - loss: 0.8331 - val_accuracy: 0.7083 - val_loss: 0.5688
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 9s 492ms/step - accuracy: 0.6982 - loss: 0.5893 - val_accuracy: 0.7500 - val_loss: 0.5907
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 416ms/step - accuracy: 0.8907 - loss: 0.4910 - val_accuracy: 0.6250 - val_loss: 0.6856
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 10s 395ms/step - accuracy: 0.7679 - loss: 0.6393 - val_accuracy: 

In [18]:
scores = model.evaluate(test_ds)

65/65 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.8272 - loss: 0.4052
